In [1]:
!pip install tensorflow

     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     -------------------- ------------------- 30.7/59.7 kB ? eta -:--:--
     -------------------- ------------------- 30.7/59.7 kB ? eta -:--:--
     -------------------------------------- 59.7/59.7 kB 454.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/50.1 kB ? eta -:--:--
     ---------------------------------------- 50.1/50.1 kB 2.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB 944.1 kB/s eta 0:06:54
   ---------------------------------------- 0.3/390.3 MB 1.8 MB/s eta 0:03:43
   ---------------------------------------- 0.7/390.3 MB 2.8 MB/s eta 0:02:22
   ---------------------------------------- 0.9/390.3 MB 3.1 MB/s eta 0:02:06
   --------------------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
df = pd.read_csv("final_refined_medical_data.csv")

# Encoding categorical variables
label_encoders = {}
categorical_columns = ['Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Gender', 'Blood Pressure', 'Cholesterol Level', 'Sugar Level', 'Blood Pressure Level']

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Encode target variables
drug_encoder = LabelEncoder()
dosage_encoder = LabelEncoder()
treatment_encoder = LabelEncoder()

df['Drug Dosage'] = dosage_encoder.fit_transform(df['Drug Dosage'])
df['Personalized Treatment'] = treatment_encoder.fit_transform(df['Personalized Treatment'])
df['Drug Names'] = drug_encoder.fit_transform(df['Drug Names'])

# Define features and targets
X = df[categorical_columns + ['Age']]
y_drug = df['Drug Names']
y_dosage = df['Drug Dosage']
y_treatment = df['Personalized Treatment']

# Split dataset
X_train, X_test, y_drug_train, y_drug_test = train_test_split(X, y_drug, test_size=0.2, random_state=42)
X_train, X_test, y_dosage_train, y_dosage_test = train_test_split(X, y_dosage, test_size=0.2, random_state=42)
X_train, X_test, y_treatment_train, y_treatment_test = train_test_split(X, y_treatment, test_size=0.2, random_state=42)

# Build Neural Network model for Drug Names prediction
model_drug = Sequential()
model_drug.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_drug.add(Dense(32, activation='relu'))
model_drug.add(Dense(len(np.unique(y_drug)), activation='softmax'))

model_drug.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_drug.fit(X_train, y_drug_train, epochs=50, batch_size=10, verbose=1)

# Build Neural Network model for Drug Dosage prediction
model_dosage = Sequential()
model_dosage.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_dosage.add(Dense(32, activation='relu'))
model_dosage.add(Dense(len(np.unique(y_dosage)), activation='softmax'))

model_dosage.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_dosage.fit(X_train, y_dosage_train, epochs=50, batch_size=10, verbose=1)

# Build Neural Network model for Personalized Treatment prediction
model_treatment = Sequential()
model_treatment.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_treatment.add(Dense(32, activation='relu'))
model_treatment.add(Dense(len(np.unique(y_treatment)), activation='softmax'))

model_treatment.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_treatment.fit(X_train, y_treatment_train, epochs=50, batch_size=10, verbose=1)

# Predictions
y_drug_pred = np.argmax(model_drug.predict(X_test), axis=1)
y_dosage_pred = np.argmax(model_dosage.predict(X_test), axis=1)
y_treatment_pred = np.argmax(model_treatment.predict(X_test), axis=1)

# Evaluate accuracy
accuracy_drug = accuracy_score(y_drug_test, y_drug_pred)
accuracy_dosage = accuracy_score(y_dosage_test, y_dosage_pred)
accuracy_treatment = accuracy_score(y_treatment_test, y_treatment_pred)

print(f"Drug Recommendation Accuracy: {accuracy_drug:.2f}")
print(f"Dosage Recommendation Accuracy: {accuracy_dosage:.2f}")
print(f"Treatment Recommendation Accuracy: {accuracy_treatment:.2f}")

# # Save models and encoders
# model_drug.save("/mnt/data/drug_prediction_model.h5")
# model_dosage.save("/mnt/data/dosage_prediction_model.h5")
# model_treatment.save("/mnt/data/treatment_prediction_model.h5")
# joblib.dump(drug_encoder, "/mnt/data/drug_encoder.pkl")
# joblib.dump(dosage_encoder, "/mnt/data/dosage_encoder.pkl")
# joblib.dump(treatment_encoder, "/mnt/data/treatment_encoder.pkl")
# joblib.dump(label_encoders, "/mnt/data/label_encoders.pkl")

# Sample input with original values
def encode_input(sample):
    for col, encoder in label_encoders.items():
        sample[col] = encoder.transform(sample[col])
    return sample

sample_input = pd.DataFrame({
    'Fever': ['Yes'],  # Yes/No
    'Cough': ['No'],  # Yes/No
    'Fatigue': ['No'],  # Yes/No
    'Difficulty Breathing': ['Yes'],  # Yes/No
    'Gender': ['Male'],  # Male/Female
    'Blood Pressure': ['High'],  # Encoded level
    'Cholesterol Level': ['Normal'],  # Encoded level
    'Sugar Level': ['High'],  # Encoded level
    'Blood Pressure Level': ['High'],  # Encoded level
    'Age': [28]  # Age in years
})

sample_input = encode_input(sample_input)

# Make predictions
predicted_drug = np.argmax(model_drug.predict(sample_input), axis=1)[0]
predicted_dosage = np.argmax(model_dosage.predict(sample_input), axis=1)[0]
predicted_treatment = np.argmax(model_treatment.predict(sample_input), axis=1)[0]

# Decode predictions
predicted_drug = drug_encoder.inverse_transform([predicted_drug])[0]
predicted_dosage = dosage_encoder.inverse_transform([predicted_dosage])[0]
predicted_treatment = treatment_encoder.inverse_transform([predicted_treatment])[0]

print(f"Recommended Drug: {predicted_drug}")
print(f"Recommended Dosage: {predicted_dosage}")
print(f"Recommended Treatment: {predicted_treatment}")


Epoch 1/50


c:\Users\MOHAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2078 - loss: 4.8282       
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3690 - loss: 1.4785 
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5986 - loss: 1.1591 
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6660 - loss: 1.1517 
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6935 - loss: 1.0151 
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7678 - loss: 0.8570 
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7797 - loss: 0.9525 
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8462 - loss: 0.8913 
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7489 - loss: 0.8646
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8283 - loss: 0.6985 
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7659 - loss: 0.9815 
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8024

c:\Users\MOHAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5095 - loss: 2.0116   
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8842 - loss: 0.7301 
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9039 - loss: 0.5593 
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9093 - loss: 0.3984 
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8829 - loss: 0.4706 
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9031 - loss: 0.4152 
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8778 - loss: 0.4918 
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8966 - loss: 0.4338 
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9025 - loss: 0.4068 
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8939 - loss: 0.3994 
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8908 - loss: 0.4209 
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8484 - 

c:\Users\MOHAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2604 - loss: 6.0217        
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8883 - loss: 0.6167 
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8996 - loss: 0.5005 
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9018 - loss: 0.4193 
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8894 - loss: 0.4705 
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9059 - loss: 0.4297 
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9111 - loss: 0.3986 
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8874 - loss: 0.3989 
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8621 - loss: 0.5155 
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8809 - loss: 0.4716 
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8866 - loss: 0.4635 
Epoch 12/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.91